In [1]:
# chargement des données
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import metrics
from scipy.stats import chi2_contingency
import scipy.stats as stats
import os
import math

import warnings
warnings.filterwarnings("ignore")

In [25]:
hackhaton=pd.read_csv("hackhaton_logic_replaced_fin.csv",encoding='ISO-8859-1')
hackhaton.drop('Unnamed: 0', axis =1, inplace=True)
hackhaton

,ID_CARRIERE,date_debut,date_fin,ETAT,CATEGORIE_EMPLOI,QUALITE,STATUT_AGENT,INDICE_BRUT,EMPLOI_STATUTAIRE,LIBELLE_EMPLOI_GRADE
0,56,01/01/1999,26/03/1999,1,A,T,T,NaN,NaN,adjointadministratif.........
1,56,27/03/1999,16/07/1999,1,A,T,T,NaN,NaN,adjointadministratif.........
2,56,17/07/1999,31/12/1999,1,A,T,T,NaN,NaN,adjointadministratif.........
3,56,01/01/2000,02/07/2000,1,A,T,T,NaN,NaN,adjointadministratif.........
4,56,03/07/2000,19/07/2000,1,A,T,T,NaN,NaN,adjointadministratif.........
...,...,...,...,...,...,...,...,...,...,...
1048570,1861885,01/02/2009,28/02/2009,1,A,T,T,303.0,NaN,adjpat2c
1048571,1861885,01/03/2009,31/03/2009,1,A,T,T,303.0,NaN,adjpat2c
1048572,1861885,01/04/2009,08/04/2009,1,A,T,T,303.0,NaN,adjpat2c
1048573,1861885,09/04/2009,19/04/2009,1,A,T,T,303.0,NaN,adjpat2c


# Normalisation des libellés (pas de caractères spéciaux, ni d'espaces, ni de voyelles)

In [27]:
hackhaton2=hackhaton


In [4]:
hackhaton2

,ID_CARRIERE,date_debut,date_fin,ETAT,CATEGORIE_EMPLOI,QUALITE,STATUT_AGENT,INDICE_BRUT,EMPLOI_STATUTAIRE,LIBELLE_EMPLOI_GRADE
0,56,01/01/1999,26/03/1999,1,A,T,T,NaN,NaN,adjointadministratif.........
1,56,27/03/1999,16/07/1999,1,A,T,T,NaN,NaN,adjointadministratif.........
2,56,17/07/1999,31/12/1999,1,A,T,T,NaN,NaN,adjointadministratif.........
3,56,01/01/2000,02/07/2000,1,A,T,T,NaN,NaN,adjointadministratif.........
4,56,03/07/2000,19/07/2000,1,A,T,T,NaN,NaN,adjointadministratif.........
...,...,...,...,...,...,...,...,...,...,...
1048570,1861885,01/02/2009,28/02/2009,1,A,T,T,303.0,NaN,adjpat2c
1048571,1861885,01/03/2009,31/03/2009,1,A,T,T,303.0,NaN,adjpat2c
1048572,1861885,01/04/2009,08/04/2009,1,A,T,T,303.0,NaN,adjpat2c
1048573,1861885,09/04/2009,19/04/2009,1,A,T,T,303.0,NaN,adjpat2c


In [28]:
hackhaton2['LIBELLE_EMPLOI_GRADE']=[str(i) for i in hackhaton2['LIBELLE_EMPLOI_GRADE']]

In [29]:
#on enleve les voyelles des libellés ? (si jamais il y a des abreviations, souvent on saute les voyelles...) 
#On enleve aussi les espaces on sait jamais

voyelles='aeiouyÃÂ'

L=[]
for i in hackhaton2['LIBELLE_EMPLOI_GRADE'] :
    
    if i != 'nan' :
        m=''
        
        for j in range(0, len(i)):
            
            if i[j] in voyelles:  #Enlever les voyelles (pour les abréviations) 
                pass
            
            else :
                m = m + i[j]
        
        
    else :
        m=i
    
    m = "".join(char for char in m if char.isalnum()) #Enlever les caractères spéciaux
    L.append(m)

In [30]:
hackhaton2['LIBELLE_EMPLOI_GRADE']=[str(i) for i in L]

In [8]:
hackhaton2

,ID_CARRIERE,date_debut,date_fin,ETAT,CATEGORIE_EMPLOI,QUALITE,STATUT_AGENT,INDICE_BRUT,EMPLOI_STATUTAIRE,LIBELLE_EMPLOI_GRADE
0,56,01/01/1999,26/03/1999,1,A,T,T,NaN,NaN,djntdmnstrtf
1,56,27/03/1999,16/07/1999,1,A,T,T,NaN,NaN,djntdmnstrtf
2,56,17/07/1999,31/12/1999,1,A,T,T,NaN,NaN,djntdmnstrtf
3,56,01/01/2000,02/07/2000,1,A,T,T,NaN,NaN,djntdmnstrtf
4,56,03/07/2000,19/07/2000,1,A,T,T,NaN,NaN,djntdmnstrtf
...,...,...,...,...,...,...,...,...,...,...
1048570,1861885,01/02/2009,28/02/2009,1,A,T,T,303.0,NaN,djpt2c
1048571,1861885,01/03/2009,31/03/2009,1,A,T,T,303.0,NaN,djpt2c
1048572,1861885,01/04/2009,08/04/2009,1,A,T,T,303.0,NaN,djpt2c
1048573,1861885,09/04/2009,19/04/2009,1,A,T,T,303.0,NaN,djpt2c


# Fonction de ressemblance des libellés

## Séparation des libellés associés a des nan et les autres 

In [43]:
# liste des libellés associés a des 'nan'
ind_code_nan=[i for i in range (0,1048574) if str(hackhaton2.EMPLOI_STATUTAIRE[i]) == 'nan']
libelle_nan =[hackhaton2.LIBELLE_EMPLOI_GRADE[i] for i in ind_code_nan]
prout=libelle_nan
libelle_nan=list(set(libelle_nan))
len(libelle_nan)

7831

In [31]:
# liste des libellés  (unique) associés a des codes (dictionnaire ?)

ind_code_no_nan=[i for i in range (0,1048574) if str(hackhaton2.EMPLOI_STATUTAIRE[i]) != 'nan']
libelle_no_nan =[hackhaton2.LIBELLE_EMPLOI_GRADE[i] for i in ind_code_no_nan]
libelle_no_nan=list(set(libelle_no_nan))
len(libelle_no_nan)

7501

In [52]:
Liste=[]
for lib in libelle_no_nan : 
    Liste.append((lib,list(np.unique(list(hackhaton2[hackhaton2.LIBELLE_EMPLOI_GRADE == lib]['EMPLOI_STATUTAIRE'])))))
    

In [62]:
p=[(a,b) for a,b in dict(Liste).items() if ((len(b)==2) and ('nan' in b)) or (len(b)==1)]
p2=[(a,b[0]) if b[0]!='nan' else (a,b[1]) for a,b in p]

In [67]:
lib_to_onecode=[a for a,b in p2]

In [68]:
lib_to_onecode

['gstnnrdprsnnl',
 'cndctrspdxnv',
 'tchncnsphsptlrprncpl',
 'gntspcclsmtplcl',
 'ttchdmnhspt',
 'djntdmnstrtftrrtrld2mclss01012017',
 'xlrdsnsprncpld2mclss',
 'djttcpr1',
 'djtnttchnqprnc1cl',
 'rprpphrmhspcs',
 'djnttchnppl2mcl',
 'cdr',
 'djntdmnpl1cl',
 'mnktrdctr',
 'nfrmrblcpclsp',
 'gstndsmrchspblcs',
 'grdn1rchln',
 'rgthrptcdrdsntprmdcl',
 'gntspclsd1rclss',
 'gntspclscmtrnl',
 'rspnsblrssrcshmn',
 'dcpljnnfntn',
 'mdcn1clss',
 'sxlrprcltrclssspr',
 'dsgnntdclsprr',
 'gntdntrtnqlfc',
 'djntdmprn1cl',
 'ssstntsclsttt',
 'dclsssprrctgb',
 'prcdrsprrsnt',
 'nfrmrdplmsdtt',
 'ttchprncplddmhsp',
 'scrtrmdcltsclcxcdmprs',
 'nfrmrspschtrqcn',
 'grdchmptrchfgrdch',
 'ssstntcnsppl1rclss',
 'gntdsrvcntrr2mct',
 'sprtdctrpsprncpld1rc',
 'cndctr2nv',
 'gntdptrmnmcl',
 'nfpclsprr',
 'dfsntfnctnscdrdsntnf',
 'crsrprdtssngblg',
 'djntdmnstrtfhsptlrprncpl2clss',
 'xsnsprncpl2cl',
 'chrgdprjttdstdvlppmnt',
 'gtntrtq',
 'gntsrvcspcsvrts',
 'ssstscdcppl',
 'djntdmpl2clss',
 'dg150400mllhb',
 'dr

## Fonction de ressemblance

In [72]:
import difflib


#L=[corresondant de libelle_nan[i], si pas de correspondant, c'est libelle_nan[i]]
# de longueur len(libelle_nan) 
c1=0
c2=0
L_replacable=[]
L=[]
for i in range (0,len(libelle_nan)) :
    
    print(i+1,'/',len(libelle_nan))
    Match=difflib.get_close_matches(libelle_nan[i], lib_to_onecode, cutoff = 0.95) #On selectionne les libellé assigné a un code qui match a au moins 90% avec notre libellé sans code 

    if len(Match) != 0:
        c1=c1+1
        Match_ratio = [difflib.SequenceMatcher(None, libelle_nan[i], a).ratio() for a in Match]
        index_max= Match_ratio.index(max(Match_ratio))
        L_replacable.append(libelle_nan[i])
        L.append(Match[index_max]) #Match[index_max] est l'element le plus ressemblant a notre libelle de base
        
    else : 
        c2=c2+1
        L.append(libelle_nan[i])

1 / 7831
2 / 7831
3 / 7831
4 / 7831
5 / 7831
6 / 7831
7 / 7831
8 / 7831
9 / 7831
10 / 7831
11 / 7831
12 / 7831
13 / 7831
14 / 7831
15 / 7831
16 / 7831
17 / 7831
18 / 7831
19 / 7831
20 / 7831
21 / 7831
22 / 7831
23 / 7831
24 / 7831
25 / 7831
26 / 7831
27 / 7831
28 / 7831
29 / 7831
30 / 7831
31 / 7831
32 / 7831
33 / 7831
34 / 7831
35 / 7831
36 / 7831
37 / 7831
38 / 7831
39 / 7831
40 / 7831
41 / 7831
42 / 7831
43 / 7831
44 / 7831
45 / 7831
46 / 7831
47 / 7831
48 / 7831
49 / 7831
50 / 7831
51 / 7831
52 / 7831
53 / 7831
54 / 7831
55 / 7831
56 / 7831
57 / 7831
58 / 7831
59 / 7831
60 / 7831
61 / 7831
62 / 7831
63 / 7831
64 / 7831
65 / 7831
66 / 7831
67 / 7831
68 / 7831
69 / 7831
70 / 7831
71 / 7831
72 / 7831
73 / 7831
74 / 7831
75 / 7831
76 / 7831
77 / 7831
78 / 7831
79 / 7831
80 / 7831
81 / 7831
82 / 7831
83 / 7831
84 / 7831
85 / 7831
86 / 7831
87 / 7831
88 / 7831
89 / 7831
90 / 7831
91 / 7831
92 / 7831
93 / 7831
94 / 7831
95 / 7831
96 / 7831
97 / 7831
98 / 7831
99 / 7831
100 / 7831
101 / 78

In [73]:
print(c1 , 'de remplacés et ', c2, 'non remplacés sur en tout', c1+c2, 'libellés')

s = 0
for i in L_replacable : 
    s += prout.count(i)
print(s)

2730 de remplacés et  5101 non remplacés sur en tout 7831 libellés
140246


In [76]:
print((s/len(prout))*100, '% remplaçable

36.34775622335394

In [13]:
len(libelle_nan) == len(L) 
#Les deux listes pour creer le dictionnaire de correspondance entre libellé avec des nan et leur libellé (qui ressemble a 90% et avec un code) correspondant

True

In [14]:
correspondance=[{libelle_nan[i] : L[i]} for i in range (0,len(L))]

In [15]:
c=correspondance[0]
for i in range (1,len(correspondance)): 
    c.update(correspondance[i])


In [47]:
c

{'': '',
 'gstnnrdprsnnl': 'gstnnrdprsnnl',
 'ssstntcnsptrcln': 'ssstntcnsptrcln',
 'tchncnsphsptlrprncpl': 'tchncnsphsptlrprncpl',
 'tchncndlbrtr3': 'tchncndlbrtr',
 'gntsrvhsptlr': 'gntsrvhsptlr',
 'dsgnntclsssp': 'dsgnntclsssp',
 'ttchdmnhspt': 'ttchdmnhspt',
 'xlrdsnsprncpld2mclss': 'xlrdsnsprncpld2mclss',
 'djttcpr1': 'djttcpr1',
 'djdmprnc2hmclss': 'djdmprnc2mclss',
 'sgnntclxc': 'sgnntclxc',
 'tchncnlbrtcn': 'tchncnlbrtcn',
 'mssrkncnrm': 'mssrkncnrm',
 'dsgnntdcs': 'dsgnntdcs',
 'gnttchnqqlftrrtrl': 'gnttchnqqlftrrtrl',
 'djnttchnppl2mcl': 'djnttchnppl2mcl',
 'gntdmnstrtftrrtr': 'gntdmnstrtftrrtr',
 'nfrmrblcpclsp': 'nfrmrblcpclsp',
 'lvnfrmrdpschtr': 'lvnfrmrdpschtr',
 'dctrjnsnftscn': 'dctrjnsnftscn',
 'gntspclsd1rclss': 'gntspclsd1rclss',
 'gntspclscmtrnl': 'gntspclscmtrnl',
 'rspnsblrssrcshmn': 'rspnsblrssrcshmn',
 'shqlf2mctg': 'shqlf2mctgr',
 'djdmnprncpl2mclss': 'djtdmnprncpl2mclss',
 'drctrsrvcstchnqs': 'drctrsrvcstchnqs',
 'tchsphspprnc': 'tchsphspprnc',
 'cndctrsdngns

## Application a la BDD

In [17]:
def replace(i, data):
    try:
        return data[i]
    except KeyError :
        return i

In [18]:
new_grade=[replace(i,c) for i in list(hackhaton2.LIBELLE_EMPLOI_GRADE)]

In [19]:
hackhaton2.LIBELLE_EMPLOI_GRADE=new_grade

In [20]:
hackhaton2

,ID_CARRIERE,date_debut,date_fin,ETAT,CATEGORIE_EMPLOI,QUALITE,STATUT_AGENT,INDICE_BRUT,EMPLOI_STATUTAIRE,LIBELLE_EMPLOI_GRADE
0,56,01/01/1999,26/03/1999,1,A,T,T,NaN,NaN,djntdmnstrtf
1,56,27/03/1999,16/07/1999,1,A,T,T,NaN,NaN,djntdmnstrtf
2,56,17/07/1999,31/12/1999,1,A,T,T,NaN,NaN,djntdmnstrtf
3,56,01/01/2000,02/07/2000,1,A,T,T,NaN,NaN,djntdmnstrtf
4,56,03/07/2000,19/07/2000,1,A,T,T,NaN,NaN,djntdmnstrtf
...,...,...,...,...,...,...,...,...,...,...
1048570,1861885,01/02/2009,28/02/2009,1,A,T,T,303.0,NaN,djpt2c
1048571,1861885,01/03/2009,31/03/2009,1,A,T,T,303.0,NaN,djpt2c
1048572,1861885,01/04/2009,08/04/2009,1,A,T,T,303.0,NaN,djpt2c
1048573,1861885,09/04/2009,19/04/2009,1,A,T,T,303.0,NaN,djpt2c


# Remplacement des codes si égalité des libellés 

In [21]:
ind_code_nan=[i for i in range (0,1048574+1) if str(hackhaton2.EMPLOI_STATUTAIRE[i]) == 'nan']
libelle_nan =[hackhaton2.LIBELLE_EMPLOI_GRADE[i] for i in ind_code_nan] #le premier libellés correspond à celui de la ligne ind_code_nan[i]
libelle_nan=list(set(libelle_nan))
len(libelle_nan)

7051

In [22]:
ind_code_no_nan=[i for i in range (0,1048574+1) if str(hackhaton2.EMPLOI_STATUTAIRE[i]) != 'nan']
libelle_no_nan =[hackhaton2.LIBELLE_EMPLOI_GRADE[i] for i in ind_code_no_nan] #le premier libellés correspond à celui de la ligne ind_code_no_nan[i]
libelle_no_nan_unic=list(set(libelle_no_nan))
print('len(libelle_no_nan) =',len(libelle_no_nan))
print('len(libelle_no_nan_unic) =',len(libelle_no_nan_unic))

len(libelle_no_nan) = 662729
len(libelle_no_nan_unic) = 7501


In [23]:
a=list(hackhaton2.LIBELLE_EMPLOI_GRADE)

for i in range (0,len(libelle_nan)) :
    
        print(i+1,'/',len(libelle_nan))
        #print(libelle_nan[i])
        Match=difflib.get_close_matches(libelle_nan[i], libelle_no_nan_unic, cutoff = 1) #On selectionne les libellé assigné a un code qui est égal à notre libellé sans code 
        #print(Match)
        #print(libelle_nan[i] ,'match avec', Match[0])
        #print(len(Match))
        
        if len(Match) != 0 and Match[0]!='':
            
            #print(libelle_nan[i] ,'match avec', Match[0]) 
            position_nan =list(np.where(np.array(a) == libelle_nan[i])[0]) #position du libellé ou le code est nan dans hackhaton 
            #print('position des nan dans hackaton', position_nan)
            
            index_no_nan=ind_code_no_nan[libelle_no_nan.index(Match[0])] #cet index correspond au libellés de la ligne ind_code_no_nan[i]
            for j in position_nan : 
                if hackhaton2.EMPLOI_STATUTAIRE[j] != hackhaton2.EMPLOI_STATUTAIRE[index_no_nan]: 
                    
                    #print('on remplace', hackhaton.EMPLOI_STATUTAIRE[j], 'par', hackhaton2.EMPLOI_STATUTAIRE[index_no_nan])
                    hackhaton2.EMPLOI_STATUTAIRE[j]= hackhaton2.EMPLOI_STATUTAIRE[index_no_nan]

        else : 
            pass

1 / 7051
2 / 7051
3 / 7051
4 / 7051
5 / 7051
6 / 7051
7 / 7051
8 / 7051
9 / 7051
10 / 7051
11 / 7051
12 / 7051
13 / 7051
14 / 7051
15 / 7051
16 / 7051
17 / 7051
18 / 7051
19 / 7051
20 / 7051
21 / 7051
22 / 7051
23 / 7051
24 / 7051
25 / 7051
26 / 7051
27 / 7051
28 / 7051
29 / 7051
30 / 7051
31 / 7051
32 / 7051
33 / 7051
34 / 7051
35 / 7051
36 / 7051
37 / 7051
38 / 7051
39 / 7051
40 / 7051
41 / 7051
42 / 7051
43 / 7051
44 / 7051
45 / 7051
46 / 7051
47 / 7051
48 / 7051
49 / 7051
50 / 7051
51 / 7051
52 / 7051
53 / 7051
54 / 7051
55 / 7051
56 / 7051
57 / 7051
58 / 7051
59 / 7051
60 / 7051
61 / 7051
62 / 7051
63 / 7051
64 / 7051
65 / 7051
66 / 7051
67 / 7051
68 / 7051
69 / 7051
70 / 7051
71 / 7051
72 / 7051
73 / 7051
74 / 7051
75 / 7051
76 / 7051
77 / 7051
78 / 7051
79 / 7051
80 / 7051
81 / 7051
82 / 7051
83 / 7051
84 / 7051
85 / 7051
86 / 7051
87 / 7051
88 / 7051
89 / 7051
90 / 7051
91 / 7051
92 / 7051
93 / 7051
94 / 7051
95 / 7051
96 / 7051
97 / 7051
98 / 7051
99 / 7051
100 / 7051
101 / 70


KeyboardInterrupt

